In [ ]:

import rospy
import rospkg
import numpy
import plotly.graph_objects as go
import plotly.colors
import os
import json
from plotly.subplots import make_subplots
import time

NAME = "FULL_NAME"

def load_experiment_data():
    """Carga datos desde el archivo JSON usando rutas de ROS"""
    rospack = rospkg.RosPack()
    dataset_folder = rospack.get_path("neural_network") + "/handwritten_digits/"
    output_folder = rospack.get_path("neural_network") + "/experiment_results/"
    
    
    json_path = os.path.join(output_folder, 'experiment_results.json')
    output_dir = os.path.join(output_folder, 'plots')
    
    
    if not os.path.exists(json_path):
        rospy.logerr(f"Archivo JSON no encontrado en: {json_path}")
        return None, output_dir
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
        return data, output_dir
    except Exception as e:
        rospy.logerr(f"Error cargando JSON: {str(e)}")
        return None, output_dir


def create_plots(data, output_dir):
    
    architectures = numpy.array([d['architecture'] for d in data])
    lr = numpy.array([float(d['learning_rate']) for d in data])
    bs = numpy.array([int(d['batch_size']) for d in data])
    epochs = numpy.array([int(d['epochs']) for d in data])
    success = numpy.array([float(d['success_rate']) for d in data])
    times = numpy.array([float(d['training_time']) for d in data])

    # --------------------------------------------
    # 1. Análisis comparativo por arquitectura
    # --------------------------------------------
    
    fig1 = go.Figure()
    unique_arch = numpy.unique(architectures)
    
    for arch in unique_arch:
        mask = architectures == arch
        fig1.add_trace(go.Box(
            y=success[mask],
            name=arch,
            boxpoints='all',
            jitter=0.3,
            marker_color=numpy.random.choice(plotly.colors.qualitative.Plotly),
            hovertext=[
                f"LR: {lr:.2f}, BS: {bs}, E: {e}<br>Time: {t:.1f}s" 
                for lr, bs, e, t in zip(lr[mask], bs[mask], epochs[mask], times[mask])
            ]
        ))
    
    fig1.update_layout(
        title='Comparación de Accuracy entre Arquitecturas',
        yaxis_title='Accuracy (%)',
        xaxis_title='Arquitectura',
        template='plotly_white'
    )
    fig1.write_html(os.path.join(output_dir, 'architecture_comparison.html'))
    

    fig = go.Figure()
    for arch in numpy.unique(architectures):
        mask = architectures == arch
        fig.add_trace(go.Bar(
            x=[arch],
            y=[numpy.mean(success[mask])],
            error_y=dict(type='data', array=[numpy.std(success[mask])]),
            name=arch,
            text=[
                f"Promedio: {numpy.mean(success[mask]):.1f}%<br>"
                f"Mejor: {numpy.max(success[mask]):.1f}%<br>"
                f"Peor: {numpy.min(success[mask]):.1f}%"
            ],
            hoverinfo='text'
        ))
    fig.update_layout(
        title='Comparación de Accuracy por Arquitectura',
        xaxis_title='Arquitectura',
        yaxis_title='Accuracy (%)',
        template='plotly_white'
    )
    fig.write_html(os.path.join(output_dir, 'architecture_accuracy.html'))

    # --------------------------------------------
    # 2. Análisis por parámetro individual
    # --------------------------------------------
    params = {
        'learning_rate': (lr, "Tasa de Aprendizaje"),
        'batch_size': (bs, "Tamaño de Lote"), 
        'epochs': (epochs, "Épocas")
    }
    
    for param, (values, name) in params.items():
        fig = go.Figure()
        
        # Agrupamiento inteligente para parámetros continuos
        if param == 'learning_rate':
            bins = numpy.linspace(min(values), max(values), 5)
            groups = numpy.digitize(values, bins)
            labels = [f"{bins[i]:.3f}-{bins[i+1]:.3f}" for i in range(len(bins)-1)]
        else:
            groups = values
            labels = sorted(numpy.unique(groups))
        
        # Calcular métricas
        means = [numpy.mean(success[groups == g]) for g in numpy.unique(groups)]
        stds = [numpy.std(success[groups == g]) for g in numpy.unique(groups)]
        counts = [numpy.sum(groups == g) for g in numpy.unique(groups)]
        
        fig.add_trace(go.Bar(
            x=labels,
            y=means,
            error_y=dict(type='data', array=stds),
            marker_color=plotly.colors.sequential.Viridis,
            text=[f"Muestras: {c}<br>σ: {s:.1f}" for c,s in zip(counts,stds)],
            hoverinfo='text'
        ))
        
        fig.update_layout(
            title=f'Accuracy vs {name}',
            xaxis_title=name,
            yaxis_title='Accuracy (%)',
            template='plotly_white'
        )
        fig.write_html(os.path.join(output_dir, f'accuracy_vs_{param}.html'))

    # --------------------------------------------
    # 3. Análisis multivariado con coordenadas paralelas
    # --------------------------------------------
    fig = go.Figure(data=go.Parcoords(
        line=dict(
            color=success,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Accuracy')
        ),
        dimensions=[
            dict(label='Learning Rate', values=lr),
            dict(label='Batch Size', values=bs),
            dict(label='Épocas', values=epochs),
            dict(label='Accuracy', values=success),
            dict(label='Tiempo (s)', values=times)
        ]
    ))
    fig.update_layout(
        title='Análisis Multivariado de Hiperparámetros',
        template='plotly_white'
    )
    fig.write_html(os.path.join(output_dir, 'multivariate_analysis.html'))

    # --------------------------------------------
    # 4. Mapas de calor para combinaciones de parámetros
    # --------------------------------------------
    from itertools import product
    
    # Combinaciones LR-BS
    lr_vals = numpy.sort(numpy.unique(lr))
    bs_vals = numpy.sort(numpy.unique(bs))
    
    accuracy_matrix = numpy.full((len(bs_vals), len(lr_vals)), numpy.nan)
    time_matrix = numpy.full_like(accuracy_matrix, numpy.nan)
    
    for i, bs_val in enumerate(bs_vals):
        for j, lr_val in enumerate(lr_vals):
            mask = (bs == bs_val) & (lr == lr_val)
            if numpy.any(mask):
                accuracy_matrix[i,j] = numpy.mean(success[mask])
                time_matrix[i,j] = numpy.mean(times[mask])
    
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        x=lr_vals,
        y=bs_vals,
        z=accuracy_matrix,
        colorscale='Viridis',
        colorbar=dict(title='Accuracy'),
        text=[[f"Tiempo: {time_matrix[i,j]:.1f}s" for j in range(len(lr_vals))] 
              for i in range(len(bs_vals))],
        hoverinfo='text'
    ))
    fig.update_layout(
        title='Accuracy por combinación LR-BS<br>Tamaño de texto: Épocas promedio',
        xaxis_title='Learning Rate',
        yaxis_title='Batch Size',
        template='plotly_white'
    )
    fig.write_html(os.path.join(output_dir, 'lr_bs_heatmap.html'))

    # --------------------------------------------
    # 5. Análisis de tiempo de entrenamiento
    # --------------------------------------------
    fig = go.Figure()
    for arch in numpy.unique(architectures):
        mask = architectures == arch
        fig.add_trace(go.Box(
            y=times[mask],
            name=arch,
            boxpoints='all',
            jitter=0.3,
            marker_color=plotly.colors.qualitative.Plotly[numpy.random.randint(10)],
            hovertext=[
                f"LR: {lr:.2f}, BS: {bs}, E: {e}<br>Accuracy: {s:.1f}%" 
                for lr, bs, e, s in zip(lr[mask], bs[mask], epochs[mask], success[mask])
            ]
        ))
    
    fig.update_layout(
        title='Distribución de Tiempos por Arquitectura',
        yaxis_title='Tiempo (s)',
        xaxis_title='Arquitectura',
        template='plotly_white'
    )
    fig.write_html(os.path.join(output_dir, 'training_time_distribution.html'))

def main():
    rospy.init_node("nn_training")
    
    data, output_dir = load_experiment_data()
    if not data:
        rospy.signal_shutdown("Error cargando datos")
        return
    
    
    os.makedirs(output_dir, exist_ok=True)
    
    
    try:
        create_plots(data, output_dir)
        rospy.loginfo(f"Visualizaciones guardadas en: {output_dir}")
    except Exception as e:
        rospy.logerr(f"Error generando gráficas: {str(e)}")
    
    rospy.signal_shutdown("Visualización completada")


main()

[INFO] [1745352288.426862]: Visualizaciones guardadas en: /home/carloscmora/fi/robots/Mobile-Robots-2025-2/catkin_ws/src/vision/neural_network/experiment_results/plots
